In [3]:
import pandas as pd 
import numpy as np

In [4]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('expand_frame_repr', False)  # 即“禁止换行”
# pd.set_option('display.precision', 2) #展示两位小数点
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [5]:
jy_df=pd.read_csv('data/账户交易信息.csv')
jy_df.columns = ['交易流水序号','账户代号','对方账号','借贷标志','交易金额','交易余额','对方行号','交易日期','交易时间','交易渠道','摘要代号','对方名称长度']

In [6]:
agg_func = {
    '对方账号': ['nunique','count'],
    '对方行号': ['nunique'],
    '交易渠道': ['nunique'],
    '摘要代号': ['nunique'],
    '交易金额': ['sum','mean','max','min','std',np.ptp],
    '交易余额': ['sum','mean','max','min','std',np.ptp],
    '对方名称长度': ['mean','max','min','std',np.ptp],
}

In [7]:
gogo = jy_df.groupby('账户代号').agg(agg_func) ;

In [8]:
type(gogo)

pandas.core.frame.DataFrame

In [17]:
gogo.head()

对方账号          对方行号    交易渠道    摘要代号       交易金额                                                       交易余额                                                对方名称长度                  
                 nunique count nunique nunique nunique        sum     mean       max      min      std       ptp         sum      mean       max    min       std       ptp   mean max min   std ptp
账户代号                                                                                                                                                                                                
00037295453A928A      25   148      14       6      14  504204.04  3406.78  24850.00 -4600.00  3654.08  29450.00   543785.63   3674.23  24923.49   0.00   3453.20  24923.49  20.77  45   0 14.54  45
0004CC075464D54B      20    37      13       5       8  311154.50  8409.58  50000.00     0.50 12018.85  49999.50   565755.88  15290.70  50118.36   0.36  14996.75  50118.00  13.38  33   6  7.23  27
000AA77144DC1BCC      45   738      14       8      15  231726.95   313.99  13469.46    -2.00  1054.13  13471.46  1049790.98   1422.48  14057.47   0.46   2021.37  14057.01  26.82  45   0 15.84  45
001ABBF3373AFC5B      15    31       9       4       6   34718.00  1119.94   3507.00     1.00  1017.30   3506.00    36645.37   1182.11   4395.21 172.47   1106.76   4222.74  11.13  45   0 13.51  45
00310769938BC172      17    39      10       4       4 3551462.50 91063.14 200000.00  5430.00 54716.40 194570.00 16285262.27 417570.83 914599.85 818.51 223921.89 913781.34   8.46   9   6  1.17   3

In [18]:
gogo1 = gogo.reset_index() ;

In [22]:
gogo1.head()

账户代号    对方账号          对方行号    交易渠道    摘要代号       交易金额                                                       交易余额                                                对方名称长度                  
                    nunique count nunique nunique nunique        sum     mean       max      min      std       ptp         sum      mean       max    min       std       ptp   mean max min   std ptp
0  00037295453A928A      25   148      14       6      14  504204.04  3406.78  24850.00 -4600.00  3654.08  29450.00   543785.63   3674.23  24923.49   0.00   3453.20  24923.49  20.77  45   0 14.54  45
1  0004CC075464D54B      20    37      13       5       8  311154.50  8409.58  50000.00     0.50 12018.85  49999.50   565755.88  15290.70  50118.36   0.36  14996.75  50118.00  13.38  33   6  7.23  27
2  000AA77144DC1BCC      45   738      14       8      15  231726.95   313.99  13469.46    -2.00  1054.13  13471.46  1049790.98   1422.48  14057.47   0.46   2021.37  14057.01  26.82  45   0 15.84  45
3  001ABBF3373AFC5B      15    31       9       4       6   34718.00  1119.94   3507.00     1.00  1017.30   3506.00    36645.37   1182.11   4395.21 172.47   1106.76   4222.74  11.13  45   0 13.51  45
4  00310769938BC172      17    39      10       4       4 3551462.50 91063.14 200000.00  5430.00 54716.40 194570.00 16285262.27 417570.83 914599.85 818.51 223921.89 913781.34   8.46   9   6  1.17   3

In [15]:
gogo.columns

MultiIndex([(  '对方账号', 'nunique'),
            (  '对方账号',   'count'),
            (  '对方行号', 'nunique'),
            (  '交易渠道', 'nunique'),
            (  '摘要代号', 'nunique'),
            (  '交易金额',     'sum'),
            (  '交易金额',    'mean'),
            (  '交易金额',     'max'),
            (  '交易金额',     'min'),
            (  '交易金额',     'std'),
            (  '交易金额',     'ptp'),
            (  '交易余额',     'sum'),
            (  '交易余额',    'mean'),
            (  '交易余额',     'max'),
            (  '交易余额',     'min'),
            (  '交易余额',     'std'),
            (  '交易余额',     'ptp'),
            ('对方名称长度',    'mean'),
            ('对方名称长度',     'max'),
            ('对方名称长度',     'min'),
            ('对方名称长度',     'std'),
            ('对方名称长度',     'ptp')],
           )

In [16]:
gogo[('对方账号', 'nunique')]

账户代号
00037295453A928A      25
0004CC075464D54B      20
000AA77144DC1BCC      45
001ABBF3373AFC5B      15
00310769938BC172      17
0039E8B1DF35A675      14
004304374B6A674B      29
0046934E42BB38B7      11
0047FEF00AD4C1A3      41
004FF0788849679D       9
005D7A6EF2DD90C9      12
0065E0EC09A64440       5
006EFAFEBBDCCF58      11
0074FD25BBDB9009      11
0085311F2829461F      20
008575E3895D845E       3
0095DD775F7412CC      12
009E8FC00892ACD6      12
00AD928256643FB3      44
00B44B0C841BDFF3       7
00CAC587AF5B38D6      12
00D9136B872151AB      18
00DD54017F1DE2D5      13
00E0DCA30452447D      19
011F8A1AAA852DEB       8
0130A6254E5DFD05      42
01328F5215A74805      12
01334E96398278D0       6
013DB71088637BC7      10
0146ED21079D88CF      14
015470C66C936130      12
0187B03FED1591B0      10
0196043DCEACB3E0       7
0199E50555869154      41
019CA263C417C923      19
01B6A02AFAE7D514      21
01C455AE8C339942       9
01CB38B99A1491C3       9
01D8CE1F87E841DF      25
01F3AA822FA90F03    